In [1]:
import pandas as pd
import numpy as np

<h3>
2-What is the average height of the players? <h3>

In [18]:
df_home = pd.read_parquet('../data/raw/parquets/match_home_team.parquet',
                          columns=['player_id', 'height'])


df_away = pd.read_parquet('../data/raw/parquets/match_away_team.parquet',
                          columns=['player_id', 'height'])


df_all = pd.concat([df_home, df_away], ignore_index=True)


df_unique = df_all.drop_duplicates(subset='player_id')


average_height = df_unique['height'].dropna().mean()

print("Average height of all unique players:", round(average_height, 2), "cm")
print("Total unique players counted:", len(df_unique))


Average height of all unique players: 1.82 cm
Total unique players counted: 2495


<h3>
3-Which player has the highest number of wins?
<h3>

In [35]:
match_event = pd.read_parquet('../data/raw/parquets/match_event.parquet')
home_team = pd.read_parquet('../data/raw/parquets/match_home_team.parquet')
away_team = pd.read_parquet('../data/raw/parquets/match_away_team.parquet')


home_wins = match_event[match_event['winner_code'] == 1].merge(
    home_team[['match_id', 'player_id', 'full_name']],
    on='match_id',
    how='left'
)

away_wins = match_event[match_event['winner_code'] == 2].merge(
    away_team[['match_id', 'player_id', 'full_name']],
    on='match_id',
    how='left'
)


all_wins = pd.concat([home_wins[['player_id','full_name']], away_wins[['player_id','full_name']]])
win_counts = all_wins.groupby(['player_id','full_name']).size().reset_index(name='wins')


top_player = win_counts.sort_values('wins', ascending=False).head(1)
print(top_player)

      player_id              full_name  wins
1105   230049.0  Jianu, Filip Cristian    20


<h3>
4-What is the longest match recorded in terms of duration?
<h3>

In [1]:
df = pd.read_parquet("../data/raw/parquets/match_time.parquet")


period_cols = ['period_1', 'period_2', 'period_3', 'period_4', 'period_5']

df['total_duration'] = df[period_cols].sum(axis=1, skipna=True)


longest_match = df.loc[df['total_duration'].idxmax()]


print(" Longest match recorded:")
print(f"Match ID: {longest_match['match_id']}")
print(f"Total Duration: {longest_match['total_duration']} (units depend on dataset — likely seconds or minutes)")



 Longest match recorded:
Match ID: 12063611
Total Duration: 336790.0 (units depend on dataset — likely seconds or minutes)


<h3>
5-How many sets are typically played in a tennis match?
<h3>

In [4]:
df = pd.read_parquet("../data/raw/parquets/statistics_statistics.parquet")


df = df.dropna(subset=["period"])


sets_per_match = df.groupby("match_id")["period"].nunique()


average_sets = sets_per_match.mean()
average_sets = np.round(average_sets)

print(f"Average number of sets typically played in a tennis match: {average_sets:.2f}")

Average number of sets typically played in a tennis match: 3.00


<h3>
6-Which country has produced the most successful tennis players?
<h3>

In [2]:
match_event = pd.read_parquet('../data/raw/parquets/match_event.parquet')
home_team = pd.read_parquet('../data/raw/parquets/match_home_team.parquet')
away_team = pd.read_parquet('../data/raw/parquets/match_away_team.parquet')


home_wins = match_event[match_event['winner_code'] == 1].merge(
    home_team[['match_id', 'player_id', 'country']],
    on='match_id',
    how='left'
)


away_wins = match_event[match_event['winner_code'] == 2].merge(
    away_team[['match_id', 'player_id', 'country']],
    on='match_id',
    how='left'
)


all_wins = pd.concat([home_wins[['country']], away_wins[['country']]])


country_wins = all_wins.groupby('country').size().reset_index(name='wins')


top_country = country_wins.sort_values('wins', ascending=False).head(1)
print(top_country)


   country  wins
29  France   659


<h3>
7-What is the average number of aces per match?
<h3>


In [5]:
df = pd.read_parquet("../data/raw/parquets/statistics_statistics.parquet")


df_aces = df[df["statistic_name"] == "aces"]


aces_per_match = df_aces.groupby("match_id")[["home_value", "away_value"]].sum().sum(axis=1)

average_aces = aces_per_match.mean()
average_aces = np.round(average_aces)

print(f"Average number of aces per match: {average_aces:.2f}")


Average number of aces per match: 11.00
